### multiple subdomain 결과 재정렬

In [1]:
# import json
# domain = '경제'
# with open(f'evaluation/eval_data/multiple_subdomain_results/{domain}_subdomain_classified.json', 'r', encoding='utf-8') as f:
    # data = json.load(f)

#  new_data = []
# for d in data:
#     if d['qna_subdomain_reason'] in d['qna_reason']:
#         d['qna_reason'] = d['qna_reason'].replace(d['qna_subdomain_reason'], '').strip()
#         # print(d)
#     new_d = {
#         'file_id': d['file_id'],
#         'title': d['title'],
#         'chapter': d['chapter'],
#         'qna_id': d['qna_id'],
#         'qna_domain': d['qna_domain'],
#         'qna_subdomain': d['qna_subdomain'],
#         'qna_domain_reason': d['qna_reason'],
#         'qna_subdomain_reason': d['qna_subdomain_reason'],
#         'qna_question': d['qna_question'],
#         'qna_options': d['qna_options'],
#         'qna_answer': d['qna_answer'],
#         'qna_explanation': d['qna_explanation']
#     }
#     new_data.append(new_d)
    
# with open(f'evaluation/eval_data/subdomain_results_old/{domain}_subdomain_classified.json', 'w', encoding='utf-8') as f:
#     json.dump(new_data, f, ensure_ascii=False, indent=4)

## 모의고사 만들기 파이프라인

In [2]:
import os, json, random
from datetime import datetime

random.seed(42)

# 과목 분류
BBASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data'
with open(os.path.join(BBASE_DIR, 'exam_hierarchy.json'), 'r', encoding='utf-8') as f:
    subjects = json.load(f)

# subdomain 분류 완료된 데이터 > 모의고사 대상 문제주머니 추출
BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/multiple_with_subdomain'

with open(os.path.join(BASE_DIR, 'classification_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)

# 로그 파일 설정
log_output = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
log_file = f'/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs/mock_exam_pipeline_{timestamp}.log'

for json_file in os.listdir(BASE_DIR):
    if not json_file.endswith("_subdomain_classified.json"):
        continue
    
    with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    domain = json_file.replace("_subdomain_classified.json", "")
    log_message = f"=====================================\n<<< {domain} 분야 모의고사용 문제 추출 >>>"
    print(log_message)
    log_output.append(log_message)
    
    total_questions = len(data)
    if total_questions == stats[domain]['total_questions']:
        mock_data = []
        need_more = {}
        mock_total_questions = round(total_questions * 0.3)
        
        log_message1 = f"원본 문제 수: {total_questions}"
        log_message2 = f"추출 대상 문제 수: {mock_total_questions}"
        print(log_message1)
        print(log_message2)
        log_output.extend([log_message1, log_message2])

        # subdomain 별로 모의고사 만들기
        for subdomain in stats[domain]['subdomain_distribution'].keys():
            subdomain_data = [d for d in data if d['qna_subdomain'] == subdomain]
            random.shuffle(subdomain_data)
            try:
                mock_subdomain_data = random.sample(subdomain_data, mock_total_questions // len(stats[domain]['subdomain_distribution']))
            except ValueError:
                mock_subdomain_data = subdomain_data
                need_more[subdomain] = mock_total_questions // len(stats[domain]['subdomain_distribution']) - len(mock_subdomain_data)
            
            log_message = f"  - {subdomain}: {len(mock_subdomain_data)}"
            print(log_message)
            log_output.append(log_message)
            mock_data.extend(mock_subdomain_data)

        log_message1 = f" ====> 총 추출 문제: {len(mock_data)}"
        print(log_message1)
        log_output.append(log_message1)
        
        if need_more:
            log_message2 = f" ====> 모자란 문제: {need_more}"
            print(log_message2)
            log_output.append(log_message2)
    else:
        log_message = "갯수가 안맞아!"
        print(log_message)
        log_output.append(log_message)
    
    # 문제주머니 저장
    for domains in subjects.values():
        if domain in domains:
            subject = list(subjects.keys())[list(subjects.values()).index(domains)]
    
    OUTPUT_DIR = f"{BBASE_DIR}/mock_data/{subject}"

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    with open(f'{OUTPUT_DIR}/{domain}_mock_data.json', 'w', encoding='utf-8') as f:
        json.dump(mock_data, f, ensure_ascii=False, indent=4)

# 로그 파일 저장
os.makedirs('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs', exist_ok=True)
with open(log_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(log_output))

print(f"\n로그가 저장되었습니다: {log_file}")

<<< 경제 분야 모의고사용 문제 추출 >>>
원본 문제 수: 10240
추출 대상 문제 수: 3072
  - 미시경제학: 768
  - 경제정책 및 시사경제: 666
  - 거시경제학: 768
  - 국제경제학: 768
 ====> 총 추출 문제: 2970
 ====> 모자란 문제: {'경제정책 및 시사경제': 102}
<<< 보상처리 분야 모의고사용 문제 추출 >>>
원본 문제 수: 2271
추출 대상 문제 수: 681
  - 보험금심사: 227
  - 손해사정실무: 227
  - 분쟁조정 및 금융소비자보호: 227
 ====> 총 추출 문제: 681
<<< 통계 분야 모의고사용 문제 추출 >>>
원본 문제 수: 2516
추출 대상 문제 수: 755
  - 기초통계학: 188
  - 응용통계 및 시계열분석: 188
  - 추론통계학: 188
  - 데이터조사 및 분석활용: 188
 ====> 총 추출 문제: 752
<<< 리스크관리 분야 모의고사용 문제 추출 >>>
원본 문제 수: 4637
추출 대상 문제 수: 1391
  - 리스크기초이론: 347
  - 보험리스크관리: 113
  - 금융리스크측정기법: 347
  - 통합리스크관리: 228
 ====> 총 추출 문제: 1035
 ====> 모자란 문제: {'보험리스크관리': 234, '통합리스크관리': 119}
<<< 보험계약 분야 모의고사용 문제 추출 >>>
원본 문제 수: 5230
추출 대상 문제 수: 1569
  - 보험상품구조: 392
  - 언더라이팅 및 재보험: 392
  - 보험법 및 계약이론: 392
  - 보험회계 및 공시: 117
 ====> 총 추출 문제: 1293
 ====> 모자란 문제: {'보험회계 및 공시': 275}
<<< 회계 분야 모의고사용 문제 추출 >>>
원본 문제 수: 6409
추출 대상 문제 수: 1923
  - 재무회계: 641
  - 회계기준 및 감사: 410
  - 관리회계: 641
 ====> 총 추출 문제: 1692
 ====> 모자란 문제: {'회계기준 

### 수정한 도메인 원본에 반영하기?

### 분류 조정하기

In [3]:
# import os, json

# BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/multiple_with_subdomain'

# for json_file in os.listdir(BASE_DIR):
#     if not json_file.endswith("_subdomain_classified.json"):
#         continue
    
#     with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     domain = json_file.replace("_subdomain_classified.json", "")
    
#     new_data = []
#     other_domain = {}
#     for d in data:
#         if d['qna_domain'] != domain:
#             other_domain[d['qna_domain']] = d
#         else:
#             new_data.append(d)
    
#     with open(os.path.join(BASE_DIR, f'{domain}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
#         json.dump(new_data, f, ensure_ascii=False, indent=4)
        
#     for other_domain_key in other_domain.keys():
#         with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'r', encoding='utf-8') as f:
#             other_domain_data = json.load(f)
        
#         other_domain_data.append(other_domain[other_domain_key])
                
#         with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
#             json.dump(other_domain_data, f, ensure_ascii=False, indent=4)

### mock 데이터에서 모의고사 문제 추출

In [4]:
import os, json, random
from datetime import datetime

BBASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data'
BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/mock_data'
OUTPUT_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/mock_exam'

with open(os.path.join(BBASE_DIR, 'exam_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)

# 로그 파일 설정
log_output = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
log_file = f'/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs/mock_exam_extraction_{timestamp}.log'

for subject in os.listdir(BASE_DIR):
    if subject.endswith(".json"):
        continue
    
    log_message = f"=====================================\n {subject}"
    print(log_message)
    log_output.append(log_message)
    
    mock_exam_data = []
    exam_questions = 250
    for json_file in os.listdir(os.path.join(BASE_DIR, subject)):
        if not json_file.endswith("_mock_data.json"):
            continue
        
        with open(os.path.join(BASE_DIR, subject, json_file), 'r', encoding='utf-8') as f:
            json_file = json.load(f)
        
        domain = os.path.basename(f.name).replace("_mock_data.json", "")
        if domain == '디지털':
            continue
        log_message = f"-------------------------------------\n {domain}"
        print(log_message)
        log_output.append(log_message)
    
        # subdomain 별로 모의고사 만들기
        for subdomain in stats[subject][domain]['exam_subdomain_distribution'].keys():
            subdomain_data = [d for d in json_file if d['qna_subdomain'] == subdomain]
            random.shuffle(subdomain_data)
            try:
                random.seed(42)
                mock_subdomain_data = random.sample(subdomain_data, stats[subject][domain]['exam_subdomain_distribution'][subdomain])
                log_message = f" - {subdomain}: {len(mock_subdomain_data)}"
                print(log_message)
                log_output.append(log_message)
            except ValueError:
                mock_subdomain_data = subdomain_data
                log_message = f" - (ERROR) {subdomain}: {len(mock_subdomain_data)}/{stats[subject][domain]['exam_subdomain_distribution'][subdomain]}"
                print(log_message)
                log_output.append(log_message)
                
            mock_exam_data.extend(mock_subdomain_data)
        
    log_message = f" ====> 총 추출 문제: {len(mock_exam_data)}/{exam_questions} ({len(mock_exam_data)/exam_questions:.2%})"
    print(log_message)
    log_output.append(log_message)
    
    # 출력 디렉토리 생성
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    with open(f'{OUTPUT_DIR}/{subject}_mock_exam.json', 'w', encoding='utf-8') as f:
        json.dump(mock_exam_data, f, ensure_ascii=False, indent=4)

# 로그 파일 저장
os.makedirs('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs', exist_ok=True)
with open(log_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(log_output))

print(f"\n로그가 저장되었습니다: {log_file}")

 금융일반
-------------------------------------
 경영
 - 경영컨설팅 및 기술평가: 21
 - 경영학원론 및 조직관리: 21
 - 재무관리 및 기업가치평가: 21
 - 마케팅 및 영업전략: 20
-------------------------------------
 통계
 - 기초통계학: 21
 - 응용통계 및 시계열분석: 20
 - 추론통계학: 21
 - 데이터조사 및 분석활용: 21
-------------------------------------
 경제
 - 미시경제학: 21
 - 경제정책 및 시사경제: 21
 - 거시경제학: 21
 - 국제경제학: 21
 ====> 총 추출 문제: 250/250 (100.00%)
 금융실무2
-------------------------------------
 보험계약
 - 보험상품구조: 32
 - 언더라이팅 및 재보험: 30
 - 보험법 및 계약이론: 33
 - 보험회계 및 공시: 30
-------------------------------------
 보상처리
 - 보험금심사: 42
 - 손해사정실무: 42
 - 분쟁조정 및 금융소비자보호: 41
 ====> 총 추출 문제: 250/250 (100.00%)
 금융심화
-------------------------------------
 세무
 - 소득세·법인세: 21
 - 세무회계 및 재정정책: 21
 - 상속세·증여세: 20
 - 부가가치세 및 원천징수: 21
-------------------------------------
 회계
 - 재무회계: 28
 - 회계기준 및 감사: 28
 - 관리회계: 28
-------------------------------------
 노무
 - 노동법 및 사회보험: 28
 - 근로기준법 및 인사관리: 28
 - 노사관계 및 분쟁조정: 27
 ====> 총 추출 문제: 250/250 (100.00%)
 금융실무1
-------------------------------------
 내부통제
 

### multiple options 오류들

In [5]:
import json
import re

with open(os.path.join(BBASE_DIR, 'exam_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)

for subject in stats.keys():
    print("================\n", subject)
    with open(f'evaluation/eval_data/mock_exam/{subject}_mock_exam.json', 'r', encoding='utf-8') as f:
        data = json.load(f)

    # ①②③④⑤로 시작하지 않는 경우를 찾기
    invalid_options = []

    for d in data:
        if d['qna_options'] and isinstance(d['qna_options'], list):
            options = d['qna_options']
            
            # 각 옵션을 확인
            for i, option in enumerate(options):
                # ①②③④⑤로 시작하지 않는 경우 찾기
                if not re.match(r'^[①②③④⑤]', option.strip()):
                    invalid_options.append({
                        'file_id': d['file_id'],
                        'qna_domain': d['qna_domain'],
                        'qna_subdomain': d['qna_subdomain'],
                        'qna_id': d['qna_id'],
                        'option_index': i,
                        'original_option': option,
                        # 'first_char': option.strip()[0] if option.strip() else ''
                    })

    print(f"①②③④⑤로 시작하지 않는 options 수: {len(invalid_options)}")
    print(invalid_options)

 금융일반
①②③④⑤로 시작하지 않는 options 수: 0
[]
 금융심화
①②③④⑤로 시작하지 않는 options 수: 0
[]
 금융실무1
①②③④⑤로 시작하지 않는 options 수: 0
[]
 금융실무2
①②③④⑤로 시작하지 않는 options 수: 0
[]
